# Ingesting and Filtering OSM Data to GeoParquet format

This notebook automates the process of extracting specific features (you can look them up in the `extraction_tasks` list) for multiple cities (Pinerolo, Milan, Rome) from a large OpenStreetMap PBF file.

It uses `quackosm` for efficient filtering and saves the output as optimized `.geoparquet` files, ready for the benchmark scripts.

**Required libraries:** `quackosm`, `osmnx`, `pathlib`.

In [1]:
import os
import urllib.request
import quackosm
import osmnx as ox
from pathlib import Path

In [2]:
# Define Paths and URLs
# NOTE: PROJECT_ROOT is defined to display relative paths in the output, protecting user privacy.
WORKING_ROOT = Path('.').resolve().parent
PROJECT_ROOT = WORKING_ROOT.parent
RAW_DATA_DIR = WORKING_ROOT / 'data' / 'raw'
PROCESSED_DATA_DIR = WORKING_ROOT / 'data' / 'processed'

# The PBF file will be downloaded from this URL
PBF_URL = "https://download.geofabrik.de/europe/italy-latest.osm.pbf"
PBF_FILENAME = PBF_URL.split('/')[-1]
PBF_FILEPATH = RAW_DATA_DIR / PBF_FILENAME

# Ensure the required directories exist
RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

# Download the PBF file only if it's missing
if not PBF_FILEPATH.exists():
    print(f"PBF file not found. Downloading from {PBF_URL}.")
    urllib.request.urlretrieve(PBF_URL, PBF_FILEPATH)
    print(f"Download complete. File saved to: {PBF_FILEPATH.relative_to(PROJECT_ROOT)}")
else:
    print(f"PBF file already found at: {PBF_FILEPATH.relative_to(PROJECT_ROOT)}. Skipping download.")

# A list of all datasets to be generated.
# Each dictionary defines a single extraction operation for a city and feature type.
extraction_tasks = [
    # Pinerolo - Use Case 3 Data
    {'city': 'Pinerolo, Italy', 'feature_type': 'buildings', 'tags': {'building': True}},
    {'city': 'Pinerolo, Italy', 'feature_type': 'restaurants', 'tags': {'amenity': 'restaurant'}},
    {'city': 'Pinerolo, Italy', 'feature_type': 'bus_stops', 'tags': {'highway': 'bus_stop'}},

    # Pinerolo - Use Case 4 Data
    {'city': 'Pinerolo, Italy', 'feature_type': 'neighborhoods', 'tags': {'boundary': 'administrative', 'admin_level': '9'}},
    {'city': 'Pinerolo, Italy', 'feature_type': 'parks', 'tags': {'leisure': 'park'}},
    {'city': 'Pinerolo, Italy', 'feature_type': 'hospitals', 'tags': {'amenity': 'hospital'}},
    {'city': 'Pinerolo, Italy', 'feature_type': 'residential_streets', 'tags': {'highway': 'residential'}},
    {'city': 'Pinerolo, Italy', 'feature_type': 'trees', 'tags': {'natural': 'tree'}},

    # Milano - Use Case 3 Data
    {'city': 'Milan, Italy', 'feature_type': 'buildings', 'tags': {'building': True}},
    {'city': 'Milan, Italy', 'feature_type': 'restaurants', 'tags': {'amenity': 'restaurant'}},
    {'city': 'Milan, Italy', 'feature_type': 'bus_stops', 'tags': {'highway': 'bus_stop'}},

    # Milano - Use Case 4 Data
    {'city': 'Milan, Italy', 'feature_type': 'neighborhoods', 'tags': {'boundary': 'administrative', 'admin_level': '9'}},
    {'city': 'Milan, Italy', 'feature_type': 'parks', 'tags': {'leisure': 'park'}},
    {'city': 'Milan, Italy', 'feature_type': 'hospitals', 'tags': {'amenity': 'hospital'}},
    {'city': 'Milan, Italy', 'feature_type': 'residential_streets', 'tags': {'highway': 'residential'}},
    {'city': 'Milan, Italy', 'feature_type': 'trees', 'tags': {'natural': 'tree'}},

    # Roma - Use Case 3 Data
    {'city': 'Rome, Italy', 'feature_type': 'buildings', 'tags': {'building': True}},
    {'city': 'Rome, Italy', 'feature_type': 'restaurants', 'tags': {'amenity': 'restaurant'}},
    {'city': 'Rome, Italy', 'feature_type': 'bus_stops', 'tags': {'highway': 'bus_stop'}},

    # Roma - Use Case 4 Data
    {'city': 'Rome, Italy', 'feature_type': 'neighborhoods', 'tags': {'boundary': 'administrative', 'admin_level': '9'}},
    {'city': 'Rome, Italy', 'feature_type': 'parks', 'tags': {'leisure': 'park'}},
    {'city': 'Rome, Italy', 'feature_type': 'hospitals', 'tags': {'amenity': 'hospital'}},
    {'city': 'Rome, Italy', 'feature_type': 'residential_streets', 'tags': {'highway': 'residential'}},
    {'city': 'Rome, Italy', 'feature_type': 'trees', 'tags': {'natural': 'tree'}},
]

print("\nStarting data extraction process.")
tasks_completed = 0
tasks_skipped = 0

# Iterate over each task and generate the corresponding file
for task in extraction_tasks:
    city_name_query = task['city']
    # Use a clean name for the output file (e.g., "pinerolo")
    city_name_file = city_name_query.split(',')[0].lower()
    feature_type = task['feature_type']
    tags_filter = task['tags']

    output_filename = f"{city_name_file}_{feature_type}.geoparquet"
    output_filepath = PROCESSED_DATA_DIR / output_filename

    print(f"\nStarting the extraction of '{feature_type}' for '{city_name_query}'.")

    # This check is crucial to avoid re-running lengthy extractions
    if output_filepath.exists():
        print(f"Result file '{output_filename}' already exists. Skipping task.")
        tasks_skipped += 1
        continue

    try:
        # Get the geographic boundary for the city
        print(f"Fetching boundary for {city_name_query}.")
        boundary_gdf = ox.geocode_to_gdf(city_name_query)
        print("Boundary fetched successfully.")

        # Configure the PBF reader with geographic and tag filters
        pbf_reader = quackosm.PbfFileReader(
            geometry_filter=boundary_gdf.geometry.iloc[0],
            tags_filter=tags_filter,
            ignore_metadata_tags=True
        )

        # Execute the conversion from PBF to GeoDataFrame
        print(f"Starting extraction from {PBF_FILENAME}.")
        features_gdf = pbf_reader.convert_pbf_to_geodataframe(PBF_FILEPATH)
        print(f"Extraction complete. Found {len(features_gdf)} features.")

        # Save the result to a GeoParquet file
        if not features_gdf.empty:
            print(f"Saving data to {output_filename}.")
            features_gdf.to_parquet(output_filepath)
            print(f"Successfully saved file to {output_filepath.relative_to(PROJECT_ROOT)}.")
        else:
            print("Warning: No features found for this task. An empty GeoParquet file will not be created.")

        tasks_completed += 1

    except Exception as e:
        print(f"ERROR while processing task for {city_name_query}: {e}.")

print("\nData preparation process finished.")
print(f"Tasks completed successfully: {tasks_completed}")
print(f"Tasks skipped (already done): {tasks_skipped}")

PBF file already found at: UseCasesManagement\data\raw\italy-latest.osm.pbf. Skipping download.

Starting data extraction process.

Starting the extraction of 'buildings' for 'Pinerolo, Italy'.
Result file 'pinerolo_buildings.geoparquet' already exists. Skipping task.

Starting the extraction of 'restaurants' for 'Pinerolo, Italy'.
Result file 'pinerolo_restaurants.geoparquet' already exists. Skipping task.

Starting the extraction of 'bus_stops' for 'Pinerolo, Italy'.
Result file 'pinerolo_bus_stops.geoparquet' already exists. Skipping task.

Starting the extraction of 'neighborhoods' for 'Pinerolo, Italy'.
Fetching boundary for Pinerolo, Italy.
Boundary fetched successfully.
Starting extraction from italy-latest.osm.pbf.


Output()

Finished operation in 0:03:17

Output()

Extraction complete. Found 35 features.
Saving data to pinerolo_neighborhoods.geoparquet.
Successfully saved file to UseCasesManagement\data\processed\pinerolo_neighborhoods.geoparquet.

Starting the extraction of 'parks' for 'Pinerolo, Italy'.
Fetching boundary for Pinerolo, Italy.
Boundary fetched successfully.
Starting extraction from italy-latest.osm.pbf.


...

Finished operation in 0:07:57

Extraction complete. Found 26743 features.
Saving data to rome_residential_streets.geoparquet.
Successfully saved file to UseCasesManagement\data\processed\rome_residential_streets.geoparquet.

Starting the extraction of 'trees' for 'Rome, Italy'.
Fetching boundary for Rome, Italy.
Boundary fetched successfully.
Starting extraction from italy-latest.osm.pbf.


Output()

Finished operation in 0:07:25

Extraction complete. Found 35138 features.
Saving data to rome_trees.geoparquet.
Successfully saved file to UseCasesManagement\data\processed\rome_trees.geoparquet.

Data preparation process finished.
Tasks completed successfully: 15
Tasks skipped (already done): 9


In [8]:
import geopandas as gpd
from pathlib import Path

# Ensure the path points to your 'processed' folder
# Change the filename to check any other dataset
WORKING_ROOT = Path('.').resolve().parent
PROCESSED_DATA_DIR = WORKING_ROOT / 'data' / 'processed'
file_to_check = PROCESSED_DATA_DIR / 'rome_neighborhoods.geoparquet'

# Load the GeoParquet file
print(f"Inspecting file: {file_to_check.name}")
gdf = gpd.read_parquet(file_to_check)

# Print the list of columns
print("\n File Columns.")
print(gdf.columns)

# Print the first few rows to see the structure
print("\n Data Preview.")
print(gdf.head())

Inspecting file: rome_neighborhoods.geoparquet

 File Columns.
Index(['boundary', 'admin_level', 'geometry'], dtype='object')

 Data Preview.
                        boundary admin_level  \
feature_id                                     
relation/41455    administrative        None   
way/192699676     administrative        None   
relation/1460885  administrative        None   
way/553983628     administrative        None   
way/186052308     administrative        None   

                                                           geometry  
feature_id                                                           
relation/41455    POLYGON ((12.41594 41.65564, 12.41646 41.65601...  
way/192699676     LINESTRING (12.41594 41.65564, 12.41646 41.656...  
relation/1460885  POLYGON ((12.25405 41.74515, 12.25857 41.74692...  
way/553983628     LINESTRING (12.43671 41.77197, 12.43704 41.771...  
way/186052308     LINESTRING (12.45145 41.68172, 12.45167 41.681...  
